In [1]:
import os
import sys
import random
from PIL import Image
import numpy as np
import pandas as pd
#for image transform
import cv2

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import random
import time

sys.path.insert(1, '//workspace/dl2020')
#from model_loader_CP2 import *
from CP_helper_RCNN import *
from data_helper_RCNN import *

#from Unet import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data


 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
#from engine import train_one_epoch, evaluate


from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

%load_ext autoreload
%autoreload 2
%load_ext autotime

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);
image_folder = '../data'
annotation_csv = '../data/annotation.csv'

time: 4.82 ms


In [3]:
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)
train_labeled_scene_index, val_labeled_scene_index  = gen_train_val_index(labeled_scene_index)

time: 2.78 ms


In [4]:
transform = transforms.ToTensor()

kwargs = {
    #'first_dim': 'sample',
    'transform': transform,
    'image_folder': image_folder,
    'annotation_file': annotation_csv,
    'extra_info': True}

#dataset_train = LabeledDataset_RCNN (scene_index=train_labeled_scene_index, **kwargs)
#dataset_val = LabeledDataset_RCNN (scene_index=val_labeled_scene_index, **kwargs)

dataset_train = LabeledDataset(scene_index=train_labeled_scene_index, **kwargs)
dataset_val = LabeledDataset(scene_index=val_labeled_scene_index, **kwargs)





train_data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=10, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

val_data_loader = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

time: 938 ms


In [5]:
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.rpn import RPNHead
def fr50_Model(pretrained = False):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=pretrained) #true works
    # create an anchor_generator for the FPN
    # which by default has 5 outputs
    anchor_generator = AnchorGenerator(
        #sizes=tuple([(16, 32, 64, 128, 256, 512) for _ in range(5)]),
        sizes=tuple([(10, 15, 20, 30, 40) for _ in range(5)]),
         
        aspect_ratios=tuple([(0.25, 0.5, 1.0, 2.0) for _ in range(5)]))
    
    
    
    model.rpn.anchor_generator = anchor_generator

    # 256 because that's the number of features that FPN returns
    model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
    return model

time: 8.76 ms


In [6]:
#check one sample from train with batch size 20
for i, (sample, old_targets, road_image, extra) in enumerate(train_data_loader): #, print_freq, header): 
        
        images = sample
        targets = trans_target(old_targets)
        #print("images len {}, targets len {}".format(len(images), len(targets)))
        #print("images[0] shape {}".format(images[0].shape)) # [6, 3, 256, 306]      
        #images = list(image.to(device) for image in images)
        #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        break

time: 2.99 s


In [7]:
normalize = torchvision.transforms.Normalize(mean=[0.6394939, 0.6755114, 0.7049375],
                                 std=[0.31936955, 0.3117349 , 0.2953726 ])

tt = transforms.Compose([transforms.Resize((800, 800)), transforms.ToTensor(), normalize]) 

images = [tt(sew_images(s)) for s in sample] #list of [3, 800, 800], should be 1 per patch
targets = trans_target(old_targets)

time: 1.09 s


In [8]:
len(targets)

10

time: 24 ms


In [9]:
#testmodel = fr50_Model(True)

time: 1.27 ms


In [10]:
#loss_dict = testmodel(images, targets)

time: 2.05 ms


In [11]:
#loss_dict #okay works

time: 1.23 ms


In [12]:
## try training the a model with sew images non pannorm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2 #4 returned classifier as nan

# get the model using our helper function
#model = get_instance_segmentation_model(num_classes)
model = fr50_Model(False) #try False

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0001, #reduce from 0.005 to help with the classifer loss = nan issue
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

time: 5.1 s


In [13]:
num_epochs = 20

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch_FastRCNN(model, optimizer, train_data_loader, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()

Epoch: [0]  [  0/240]  eta: 0:33:10  lr: 0.000100  loss: 5.5498 (5.5498)  loss_classifier: 4.7890 (4.7890)  loss_box_reg: 0.0002 (0.0002)  loss_objectness: 0.6896 (0.6896)  loss_rpn_box_reg: 0.0709 (0.0709)  time: 8.2927  data: 2.2061  max mem: 7277
Epoch: [0]  [100/240]  eta: 0:06:53  lr: 0.000100  loss: 1.1688 (1.4701)  loss_classifier: 0.2803 (0.6157)  loss_box_reg: 0.0353 (0.0261)  loss_objectness: 0.6309 (0.6624)  loss_rpn_box_reg: 0.1713 (0.1659)  time: 3.0628  data: 0.0592  max mem: 8717
Epoch: [0]  [200/240]  eta: 0:01:58  lr: 0.000100  loss: 0.9257 (1.2053)  loss_classifier: 0.1919 (0.4050)  loss_box_reg: 0.0311 (0.0307)  loss_objectness: 0.5290 (0.6172)  loss_rpn_box_reg: 0.1612 (0.1524)  time: 3.1356  data: 0.0598  max mem: 8717


RuntimeError: CUDA out of memory. Tried to allocate 628.00 MiB (GPU 0; 10.92 GiB total capacity; 8.02 GiB already allocated; 301.00 MiB free; 2.06 GiB cached)

time: 11min 27s


In [ ]:


torch.save({
            'epoch': num_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss,
            
            }, "../models/fastRCNN_sew620epoch.pt")